#### Importing necessary libraries

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None # This avoids SettingWithCopyWarnings
import numpy as np

In [3]:
from pandas_datareader import data as pdr
import fix_yahoo_finance as yf
yf.pdr_override()

#### Initializing Variables


In [4]:
data={} # Dictionary to contain pandas dataframe for all the stocks. This is to avoid creating variable for each stock 
        # to store data
f=3 # f indicates factor/ multiplier
n=7 # Period for ATR

### Average True Range

In [5]:
def ATR(df,n): #df is the DataFrame, n is the period 7,14 ,etc
    df['H-L']=abs(df['High']-df['Low'])
    df['H-PC']=abs(df['High']-df['Close'].shift(1))
    df['L-PC']=abs(df['Low']-df['Close'].shift(1))
    df['TR']=df[['H-L','H-PC','L-PC']].max(axis=1)
    df['ATR']=np.nan
    df.ix[n-1,'ATR']=df['TR'][:n-1].mean() #.ix is deprecated from pandas version- 0.19
    for i in range(n,len(df)):
        df['ATR'][i]=(df['ATR'][i-1]*(n-1)+ df['TR'][i])/n
    return

### Supertrend is a trend-following indicator and it is plotted on prices and their placement indicates the current trend. 

In [6]:
def ST(df,f,n): #df is the dataframe, n is the period, f is the factor; f=3, n=7 are commonly used.
    #Calculation of SuperTrend
    df['Upper Basic']=(df['High']+df['Low'])/2+(f*df['ATR'])
    df['Lower Basic']=(df['High']+df['Low'])/2-(f*df['ATR'])
    df['Upper Band']=df['Upper Basic']
    df['Lower Band']=df['Lower Basic']
    for i in range(n,len(df)):
        if df['Close'][i-1]<=df['Upper Band'][i-1]:
            df['Upper Band'][i]=min(df['Upper Basic'][i],df['Upper Band'][i-1])
        else:
            df['Upper Band'][i]=df['Upper Basic'][i]    
    for i in range(n,len(df)):
        if df['Close'][i-1]>=df['Lower Band'][i-1]:
            df['Lower Band'][i]=max(df['Lower Basic'][i],df['Lower Band'][i-1])
        else:
            df['Lower Band'][i]=df['Lower Basic'][i]   
    df['SuperTrend']=np.nan
    for i in df['SuperTrend']:
        if df['Close'][n-1]<=df['Upper Band'][n-1]:
            df['SuperTrend'][n-1]=df['Upper Band'][n-1]
        elif df['Close'][n-1]>df['Upper Band'][i]:
            df['SuperTrend'][n-1]=df['Lower Band'][n-1]
    for i in range(n,len(df)):
        if df['SuperTrend'][i-1]==df['Upper Band'][i-1] and df['Close'][i]<=df['Upper Band'][i]:
            df['SuperTrend'][i]=df['Upper Band'][i]
        elif  df['SuperTrend'][i-1]==df['Upper Band'][i-1] and df['Close'][i]>=df['Upper Band'][i]:
            df['SuperTrend'][i]=df['Lower Band'][i]
        elif df['SuperTrend'][i-1]==df['Lower Band'][i-1] and df['Close'][i]>=df['Lower Band'][i]:
            df['SuperTrend'][i]=df['Lower Band'][i]
        elif df['SuperTrend'][i-1]==df['Lower Band'][i-1] and df['Close'][i]<=df['Lower Band'][i]:
            df['SuperTrend'][i]=df['Upper Band'][i]
    return


### Set of Stocks

In [7]:
StockList=['INFY', 'ICICIBANK'] 

### Fetching data using Yahoo API,  Setting up Pandas Dataframe in the dictionary 'data' with key as Stock name and Value as DataFrame


In [8]:
start='2016-1-1'
for stock in StockList:
    data[stock]=pdr.get_data_yahoo(stock+'.NS', start)

[*********************100%***********************]  1 of 1 downloaded

#### Sometimes due to rate limiting factor, Yahoo api when called yeilds no data. Below cell helps us out in figuring whether we need to ping the api again. 

In [9]:
for stock in data:
    if data[stock].empty:
        print stock

In [10]:
for stock in data:
    print stock, data[stock].head()

ICICIBANK                   Open        High         Low       Close   Adj Close  \
Date                                                                     
2016-01-01  237.544998  239.636002  234.544998  239.091003  229.999863   
2016-01-04  237.272995  237.591003  231.500000  232.317993  223.484390   
2016-01-05  232.955002  234.091003  228.817993  233.363998  224.490616   
2016-01-06  232.182007  233.408997  226.544998  227.363998  218.718781   
2016-01-07  224.000000  225.182007  221.000000  224.317993  215.788589   

              Volume  
Date                  
2016-01-01   5998097  
2016-01-04   9435792  
2016-01-05   8966978  
2016-01-06  17416181  
2016-01-07  18240713  
INFY                    Open         High          Low        Close    Adj Close  \
Date                                                                          
2016-01-01  1100.000000  1108.949951  1093.199951  1105.250000  1049.958252   
2016-01-04  1099.949951  1102.449951  1076.050049  1078.900024  1024

###  Data Cleaning, Column renaming, etc

In [11]:
for stock in data:
    data[stock].drop(data[stock][data[stock].Volume == 0].index, inplace=True) # Data Cleaning
    ATR(data[stock],n)
    data[stock]=data[stock][['Open','High','Low','Close','ATR']] # Removing unwanted columns

### SuperTrend Calculation

In [12]:
for stock in data:
    ST(data[stock],f,n)
    data[stock]=data[stock][['Open','High','Low','Close','ATR','SuperTrend']] # Removing unwanted columns
    data[stock]=data[stock].round(2)

In [13]:
data['INFY'].head() #Since ATR period is 7, we will see atr column filling out from 7th row

,Open,High,Low,Close,ATR,SuperTrend
Date,,,,,,
2016-01-01,1100.00,1108.95,1093.20,1105.25,NaN,NaN
2016-01-04,1099.95,1102.45,1076.05,1078.90,NaN,NaN
2016-01-05,1085.50,1085.50,1062.25,1074.05,NaN,NaN
2016-01-06,1074.65,1074.65,1058.90,1069.35,NaN,NaN
2016-01-07,1060.05,1067.90,1047.90,1050.80,NaN,NaN


In [14]:
data['ICICIBANK'].tail(15)

,Open,High,Low,Close,ATR,SuperTrend
Date,,,,,,
2018-03-08,290.25,298.00,289.25,296.95,9.66,317.74
2018-03-09,299.00,299.50,290.70,292.70,9.54,317.74
2018-03-12,295.60,301.90,293.75,300.65,9.49,317.74
2018-03-13,300.10,307.00,299.60,304.35,9.19,317.74
2018-03-14,303.15,307.45,297.80,306.05,9.26,317.74
2018-03-15,305.75,305.75,300.40,301.45,8.74,317.74
2018-03-16,301.25,301.85,297.40,298.10,8.13,317.74
2018-03-19,298.10,300.00,291.25,294.55,8.22,317.74
2018-03-20,293.00,297.20,290.25,292.00,8.04,317.74


### Taking data to excel

In [15]:
writer = pd.ExcelWriter('Data.xlsx')
for stock in data:
    data[stock].to_excel(writer,stock)
writer.save()